In [1]:
import os
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import backend as K
import matplotlib.pyplot as plt
from datetime import datetime
from LReLU import LReLU
import os
import numpy as np
import cv2
import random
# from getData import get_data

# import model here
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

random.seed(552)
split_path = '../data/images/split/train_val_test_crop'



Using TensorFlow backend.


In [2]:
def get_data(folder_path, input_shape):
    dim1, dim2, channels = input_shape
    
    # Load training images
    train_path = folder_path + '/train'

    x_train = []
    y_train = []
    x_test = []
    y_test = []
    x_val = []
    y_val = []

    dir_peek = os.listdir(train_path)[0]
    train_size = len(os.path.join(train_path, dir_peek))
    for dir in os.listdir(train_path):
        for file in os.listdir(os.path.join(train_path, dir)):
            img_path = os.path.join(train_path, dir, file)
            img = cv2.imread(img_path, 0)
            img = cv2.resize(img, (dim1, dim2))
            img = np.reshape(img, (dim1, dim2, 1))
            img = np.concatenate((img, img, img), axis=2)
            x_train.append(img)
            if dir == 'male':
                y_train.append(0)
            else:
                y_train.append(1)

    # Load test images
    test_path = split_path + '/test'

    for dir in os.listdir(test_path):
        for file in os.listdir(os.path.join(test_path, dir)):
            img_path = os.path.join(test_path, dir, file)
            img = cv2.imread(img_path, 0)
            img = cv2.resize(img, (dim1, dim2))
            img = np.reshape(img, (dim1, dim2, 1))
            img = np.concatenate((img, img, img), axis=2)
            x_test.append(img)
            if dir == 'male':
                y_test.append(0)
            else:
                y_test.append(1)

    # Load val images
    val_path = split_path + '/val'

    for dir in os.listdir(val_path):
        for file in os.listdir(os.path.join(val_path, dir)):
            img_path = os.path.join(val_path, dir, file)
            img = cv2.imread(img_path, 0)
            img = cv2.resize(img, (dim1, dim2))
            img = np.reshape(img, (dim1, dim2, 1))
            img = np.concatenate((img, img, img), axis=2)
            x_val.append(img)
            if dir == 'male':
                y_val.append(0)
            else:
                y_val.append(1)

    x_train = np.array(x_train)
    x_test = np.array(x_test)
    x_val = np.array(x_val)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    y_val = np.array(y_val)

    return x_train, x_val, x_test, y_train, y_val, y_test

In [5]:
# load and normalize data
input_shape = (150, 150, 3)
x_train, x_val, x_test, y_train, y_val, y_test = get_data(split_path, input_shape)

train = list(zip(x_train, y_train))
test = list(zip(x_test, y_test))
val = list(zip(x_val, y_val))
random.shuffle(train)
random.shuffle(test)
random.shuffle(val)

x_train, y_train = zip(*train)
x_train = np.array(x_train)
y_train = np.array(y_train)


x_test, y_test = zip(*test)
x_test = np.array(x_test)
y_test = np.array(y_test)

x_val, y_val = zip(*val)
x_val = np.array(x_val)
y_val = np.array(y_val)


batch_size = 128
num_classes = 2
epochs = 20

# input image dimensions
img_rows, img_cols = 926, 926

# convert data format to channel last format
x_train = x_train.reshape(x_train.shape[0], input_shape[0], input_shape[1], input_shape[2])
x_val = x_val.reshape(x_val.shape[0], input_shape[0], input_shape[1], input_shape[2])
x_test = x_test.reshape(x_test.shape[0], input_shape[0], input_shape[1], input_shape[2])
input_shape = (img_rows, img_cols, 1)


# convert class vectors to binary class matrices
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_valid = keras.utils.to_categorical(y_val, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))
y_val = np.asarray(y_val).astype('float32').reshape((-1,1))
print(x_val.shape)
print(y_val.shape)


(471, 150, 150, 3)
(471, 1)


In [6]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

base_model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
model.fit(x_train, y_train, batch_size=16, epochs=epochs, validation_data=(x_val, y_val))

Epoch 1/20
138/138 [==============================] - 68s 479ms/step - loss: 1.9147 - binary_accuracy: 0.5529 - val_loss: 0.7638 - val_binary_accuracy: 0.5754
Epoch 2/20
138/138 [==============================] - 65s 471ms/step - loss: 0.8447 - binary_accuracy: 0.5714 - val_loss: 1.0025 - val_binary_accuracy: 0.4013
Epoch 3/20
138/138 [==============================] - 68s 494ms/step - loss: 0.8718 - binary_accuracy: 0.5940 - val_loss: 0.9600 - val_binary_accuracy: 0.4183
Epoch 4/20
138/138 [==============================] - 75s 541ms/step - loss: 0.8324 - binary_accuracy: 0.5849 - val_loss: 0.8136 - val_binary_accuracy: 0.5032
Epoch 5/20
138/138 [==============================] - 68s 496ms/step - loss: 0.9218 - binary_accuracy: 0.5538 - val_loss: 1.2697 - val_binary_accuracy: 0.3715
Epoch 6/20
138/138 [==============================] - 69s 498ms/step - loss: 0.8639 - binary_accuracy: 0.5833 - val_loss: 1.0304 - val_binary_accuracy: 0.6327
Epoch 7/20
138/138 [==========================